In [33]:
%pip install tensorflow
%pip install gym
%pip install keras
%pip install keras-rl2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/52.1 kB ? eta -:--:--
     ------- -------------------------------- 10.2/52.1 kB ? eta -:--:--
     ------- -------------------------------- 10.2/52.1 kB ? eta -:--:--
     ------- -------------------------------- 10.2/52.1 kB ? eta -:--:--
     ------- -------------------------------- 10.2/52.1 kB ? eta -:--:--
     ------- -------------------------------- 10.2/52.1 kB ? eta -:--:--
     --------------- ----------------------- 20.5/52.1 kB 41.0 kB/s eta 0:00:01
     --------------- ----------------------- 20.5/52.1 kB 41.0 kB/s eta 0:00:01
     --------------- ----------------------- 20.5/52.1 kB 41.0 kB/s eta 0:00:01
     --------------- ----------------------- 20.5/52.1 kB 41.0 kB/s eta 0:00:01
     --------------- ----------------------- 

In [4]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np 
import random

In [23]:
class ShowerEnv(Env):
  def __init__(self):
    # Actions that can be taken - down, stay, up
    self.action_space = Discrete(3)
    # Temperature array
    self.observation_space = Box(low=np.array([0]), high=np.array([100]))
    # Set start temp
    self.state = 38 + random.randint(-3, 3)
    # Set shower length
    self.shower_length = 60

  def step(self, action):
    # Apply action
    self.state += action -1
    # Reduce shower length by 1 second 
    self.shower_length -= 1

    # Calc reward
    if self.state >= 37 and self.state <= 39:
      reward = 1
    else:
      reward = -1

    # Check if shower is done
    if self.shower_length <= 0:
      done = True
    else:
      done = False

    # Apply noise
    self.state += random.randint(-1,1)
    # Info placeholder
    info = {}
    # Return step information
    return self.state, reward, done, info

  def render(self):
    pass

  def reset(self):
    # Reset temp
    self.state = 38 + random.randint(-3, 3)
    # Reset length
    self.shower_length = 60
    return self.state

In [25]:
env = ShowerEnv()

c:\Users\Alex\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [18]:
env.observation_space.sample()

array([11.764443], dtype=float32)

In [26]:
episodes = 10
for episode in range(1, episodes +1):
  state = env.reset()
  done = False
  score = 0

  while not done:
    #env.render()
    action = env.action_space.sample()
    n_state, reward, done, info = env.step(action)
    score += reward
  print("Episode:{} Score: {}".format(episode, score))

Episode:1 Score: -40
Episode:2 Score: -26
Episode:3 Score: -16
Episode:4 Score: -56
Episode:5 Score: -52
Episode:6 Score: -60
Episode:7 Score: -50
Episode:8 Score: -16
Episode:9 Score: -56
Episode:10 Score: -34


In [27]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [28]:
states = env.observation_space.shape
actions = env.action_space.n

In [29]:
def build_model(states, actions):
  model = Sequential()
  model.add(Dense(24, activation='relu', input_shape=states))
  model.add(Dense(24, activation='relu'))
  model.add(Dense(actions, activation='linear'))
  return model

In [37]:
del model

In [38]:
model = build_model(states, actions)

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                48        
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 3)                 75        
                                                                 
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [39]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [40]:
def build_agent(model, actions):
  policy = BoltzmannQPolicy()
  memory = SequentialMemory(limit=50000, window_length=1)
  dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
  return dqn

In [41]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

AttributeError: 'Adam' object has no attribute '_name'

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))